In [1]:
# install huggingface and datasets
!pip install -q datasets transformers
!pip install torch
!pip install pandas
!pip install pyarrow

     |████████████████████████████████| 346 kB 32.2 MB/s 
     |████████████████████████████████| 4.2 MB 61.8 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 140 kB 76.3 MB/s 
     |████████████████████████████████| 212 kB 64.6 MB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
     |████████████████████████████████| 1.1 MB 46.5 MB/s 
     |████████████████████████████████| 596 kB 55.5 MB/s 
     |████████████████████████████████| 127 kB 18.7 MB/s 
     |████████████████████████████████| 6.6 MB 54.8 MB/s 
     |████████████████████████████████| 271 kB 56.5 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 144 kB 22.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is

In [2]:
from datasets import load_dataset, ClassLabel
from transformers import BertForTokenClassification, BertTokenizer, TrainingArguments, Trainer
import numpy as np
import torch
from sklearn import metrics


import pandas as pd
import datasets
from datasets.features import ClassLabel

In [3]:
torch.cuda.is_available()
device = torch.device("cuda")

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/e_ML4NLP2/v2.1/'

Mounted at /content/drive
ajmc  hipe2020	letemps  newseye  sonar  topres19th


In [5]:
train_path = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/fi/HIPE-2022-v2.1-newseye-train-fi.tsv'
dev_path = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/fi/HIPE-2022-v2.1-newseye-dev-fi.tsv'
test_path = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/fi/HIPE-2022-v2.1-newseye-test_allmasked-fi.tsv'

#model = 'TurkuNLP/bert-base-finnish-cased-v1'
#model = 'dbmdz/bert-base-finnish-europeana-cased'
#model = 'dbmdz/bert-base-historic-multilingual-cased'
#model = 'setu4993/LaBSE'
model = 'EMBEDDIA/finest-bert'

In [6]:
# import dataset from cloned git repo
def load_dataset(path):
    df = pd.read_csv(path, sep='\t', skip_blank_lines=False, engine='python', quoting=3)
    # error_bad_lines=False, 
    return df


tsv_train = load_dataset(train_path)
tsv_dev = load_dataset(dev_path)
tsv_test = load_dataset(test_path)

In [13]:
tsv_dev[tsv_dev.TOKEN.isnull()]

,TOKEN,NE-COARSE-LIT,NE-COARSE-METO,NE-FINE-LIT,NE-FINE-METO,NE-FINE-COMP,NE-NESTED,NEL-LIT,NEL-METO,MISC
257,None,None,None,None,None,None,None,None,None,None
529,None,None,None,None,None,None,None,None,None,None
606,None,None,None,None,None,None,None,None,None,None
1421,None,None,None,None,None,None,None,None,None,None
1573,None,None,None,None,None,None,None,None,None,None
1677,None,None,None,None,None,None,None,None,None,None
2524,None,None,None,None,None,None,None,None,None,None
3155,None,None,None,None,None,None,None,None,None,None
3780,None,None,None,None,None,None,None,None,None,None
4365,None,None,None,None,None,None,None,None,None,None


In [8]:
tsv_dev.dropna(subset=['TOKEN'])

,TOKEN,NE-COARSE-LIT,NE-COARSE-METO,NE-FINE-LIT,NE-FINE-METO,NE-FINE-COMP,NE-NESTED,NEL-LIT,NEL-METO,MISC
0,# hipe2022:document_id = dev_fi_1,None,None,None,None,None,None,None,None,None
1,# hipe2022:date = 1854-01-24,None,None,None,None,None,None,None,None,None
2,# hipe2022:language = fi,None,None,None,None,None,None,None,None,None
3,# hipe2022:dataset = newseye,None,None,None,None,None,None,None,None,None
4,# hipe2022:document_type = newspaper,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
6656,uhkaus,O,_,O,_,_,O,_,_,_
6657,olisi,O,_,O,_,_,O,_,_,_
6658,pantu,O,_,O,_,_,O,_,_,_
6659,täytäntöön,O,_,O,_,_,O,_,_,NoSpaceAfter


In [ ]:
def simple_preprocess(dataframe):
    # Add end_of_document token in df
    dataframe = dataframe.dropna(subset=['TOKEN'])

    # Filter out metadata rows beginning with #
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('#')]
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('\t')]

    #transforming nan var from Float to string to use in (***)
    dataframe.MISC = dataframe.MISC.fillna('')

    return dataframe

In [ ]:
tsv_train = simple_preprocess(tsv_train)
tsv_dev = simple_preprocess(tsv_dev)
tsv_test = simple_preprocess(tsv_test)
#tsv_train = tsv_train.reset_index()
tsv_dev.head(100)

,TOKEN,NE-COARSE-LIT,NE-COARSE-METO,NE-FINE-LIT,NE-FINE-METO,NE-FINE-COMP,NE-NESTED,NEL-LIT,NEL-METO,MISC
12,Kun,O,_,O,_,_,O,_,_,_
13,muistetaan,O,_,O,_,_,O,_,_,NoSpaceAfter
14,",",O,_,O,_,_,O,_,_,_
15,sanoo,O,_,O,_,_,O,_,_,_
16,eräs,O,_,O,_,_,O,_,_,_
...,...,...,...,...,...,...,...,...,...,...
107,murhaa,O,_,O,_,_,O,_,_,_
108,ja,O,_,O,_,_,O,_,_,_
109,saaneet,O,_,O,_,_,O,_,_,_
110,aikaan,O,_,O,_,_,O,_,_,NoSpaceAfter


In [ ]:
label_set = tsv_train['NE-COARSE-LIT'].unique()
# define the label mapping for NER
label_list = label_set.tolist()
label_list.append('_')
label_num = len(label_list)

labels = ClassLabel(num_classes=label_num, names=label_list)

labels

ClassLabel(num_classes=10, names=['O', 'B-LOC', 'B-PER', 'I-PER', 'B-HumanProd', 'I-HumanProd', 'I-LOC', 'B-ORG', 'I-ORG', '_'], id=None)

In [ ]:
def create_huggingface_file(dataframe):
    import pyarrow as pa
    import re
    from datasets import Dataset

    #creating dataset in json
    hug_out = []
    idx = 0 
    items = {'id': idx,'words':[ ], 'ner': [ ]}
    hug_out.append(items)
    for index, row in dataframe.iterrows():
        if  not re.search(r'EndOfSentence', row['MISC']):
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
        else:
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
            idx += 1
            items = {'id': idx,'words':[ ], 'ner': [ ]}
            hug_out.append(items)
    #filter hug_out out, delete items which has len(words) > 380
    #hug_out = filter(lambda x: len(x['words']) < 380, hug_out)
    #json to df
    hug_out = pd.DataFrame(hug_out)

    # delete all sentences that are too long
    #hug_out = hug_out[hug_out['words'].map(len) < 512] #why does not work? QA

    ### convert to Huggingface dataset
    hug_out = Dataset(pa.Table.from_pandas(hug_out))

    return hug_out

In [ ]:
train = create_huggingface_file(tsv_train)
val = create_huggingface_file(tsv_dev)
test = create_huggingface_file(tsv_test)

#look at training data
for i in range(10):
  print(train[i])

print(len(train))
print(len(val))
print(len(test))

{'id': 0, 'words': ['Lauantaina', 'Syyskuun', '20', 'p:nä', 'UUSI', 'SUOMETAR', '1913', '—', 'N:o', '219', '—', '7', 'Flyygeliä', 'ja', 'Pianiinoja', 'maailmankuuluisista', 'tehtaista', 'Rönisch', ',', 'Steinweg', '&', 'Ritter', 'Amerikkalaisia', 'URKUHARMOONEJA', 'Estey', 'Organ', 'C:o', 'mainiota', 'tekoa', '.'], 'ner': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'id': 1, 'words': ['Huom', '!', 'Luetteloja', 'vapaasti', '!', 'Helsingin', 'Uusi', 'Musiikkikauppa', 'K', '.', 'G', '.', 'FAZER', 'Aleksanterink', '.', '38', '.', 'Puhelin', '14', '31', '.'], 'ner': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'id': 2, 'words': ['18170', 'Venäjältä', '.'], 'ner': [0, 1, 0]}
{'id': 3, 'words': ['Pääministeri', 'Kokovtsov', 'eroaa', '?', '»', 'Golos', 'Moskvy', '»', 'kirjoittaa:', 'Täysin', 'luotettavista', 'lähteistä', 'on', 'meille', 'kerrottu', 'suurista', 'muutoksista', ',', 'jotka', 'pikaisessa', 'tulevaisuudessa', 'tu

In [ ]:
'''LOADERSCRIPT = train_path = '/content/drive/MyDrive/e_ML4NLP2/dataloader.py'
MODEL = "dbmdz/bert-base-german-europeana-cased"'''

'LOADERSCRIPT = train_path = \'/content/drive/MyDrive/e_ML4NLP2/dataloader.py\'\nMODEL = "dbmdz/bert-base-german-europeana-cased"'

In [ ]:
'''de_total = load_dataset(LOADERSCRIPT, "de")'''

'de_total = load_dataset(LOADERSCRIPT, "de")'

In [ ]:
'''train = de_total["train"]
val = de_total["validation"]
test = de_total["test"]'''

'train = de_total["train"]\nval = de_total["validation"]\ntest = de_total["test"]'

In [ ]:
'''tiny_shuffle_train =  train.shuffle(seed=42).select(range(100))
tiny_train = tiny_shuffle_train.select(range(0,80))
tiny_shuffle_test = test.shuffle(seed=42).select(range(100))
tiny_test = tiny_shuffle_test.select(range(80,100))'''

'tiny_shuffle_train =  train.shuffle(seed=42).select(range(100))\ntiny_train = tiny_shuffle_train.select(range(0,80))\ntiny_shuffle_test = test.shuffle(seed=42).select(range(100))\ntiny_test = tiny_shuffle_test.select(range(80,100))'

In [ ]:
'''tiny_shuffle_train[10]'''

'tiny_shuffle_train[10]'

In [ ]:
'''labels = train.info.features["NE_COARSE_LIT"].feature
labels'''

'labels = train.info.features["NE_COARSE_LIT"].feature\nlabels'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model)

https://huggingface.co/EMBEDDIA/finest-bert/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpc0g917v_


Downloading:   0%|          | 0.00/485k [00:00<?, ?B/s]

storing https://huggingface.co/EMBEDDIA/finest-bert/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/56e886c2cf6dfcba930b97e845acc9ad280bde5a66cd6dfea0426014c9a59e8b.935617a9c544fcab824013f6820c552b2561ea95e246d9f99e186ddac9b8bf95
creating metadata file for /root/.cache/huggingface/transformers/56e886c2cf6dfcba930b97e845acc9ad280bde5a66cd6dfea0426014c9a59e8b.935617a9c544fcab824013f6820c552b2561ea95e246d9f99e186ddac9b8bf95
https://huggingface.co/EMBEDDIA/finest-bert/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpfdhqh1vl


Downloading:   0%|          | 0.00/46.0 [00:00<?, ?B/s]

storing https://huggingface.co/EMBEDDIA/finest-bert/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/8b7befd26163f72e8110a46d0f2ade1bfee7b3fc6ed308fdf2e20f8debee7911.268a1e35e9054a0c7fb4bc185891f11122ab5e15883e3333534cdf3d76681112
creating metadata file for /root/.cache/huggingface/transformers/8b7befd26163f72e8110a46d0f2ade1bfee7b3fc6ed308fdf2e20f8debee7911.268a1e35e9054a0c7fb4bc185891f11122ab5e15883e3333534cdf3d76681112
loading file https://huggingface.co/EMBEDDIA/finest-bert/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/56e886c2cf6dfcba930b97e845acc9ad280bde5a66cd6dfea0426014c9a59e8b.935617a9c544fcab824013f6820c552b2561ea95e246d9f99e186ddac9b8bf95
loading file https://huggingface.co/EMBEDDIA/finest-bert/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/EMBEDDIA/finest-bert/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/EMBEDDIA/finest-bert

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

storing https://huggingface.co/EMBEDDIA/finest-bert/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/fb6c00e85cae7f09e35e049995692aa840e064440810b07cf9d1e4df9fab8809.59020bf6435c9bad1a5ae6edb07e5cb3adf696afdc97b518196ad24afdde8014
creating metadata file for /root/.cache/huggingface/transformers/fb6c00e85cae7f09e35e049995692aa840e064440810b07cf9d1e4df9fab8809.59020bf6435c9bad1a5ae6edb07e5cb3adf696afdc97b518196ad24afdde8014
loading configuration file https://huggingface.co/EMBEDDIA/finest-bert/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fb6c00e85cae7f09e35e049995692aa840e064440810b07cf9d1e4df9fab8809.59020bf6435c9bad1a5ae6edb07e5cb3adf696afdc97b518196ad24afdde8014
Model config BertConfig {
  "_name_or_path": "EMBEDDIA/finest-bert",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initi

In [ ]:
def tokenize_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]
    tokenized_ner = []

    for sent_words, sent_nes in zip(tokenized_words, batch["ner"]):
        tokenized_ner.append(
            [labels.str2int("O")]  # BOS symbol
            + [
                ne
                for subwords, ne in zip(sent_words, sent_nes)
                for _ in range(len(subwords))
            ]
            + [labels.str2int("O")]  # EOS symbol
        )

    # Padding with "O"
    tokenized["labels"] = [(ner + [labels.str2int("O")] * (100 - len(ner)))[:100] for ner in tokenized_ner]
    tokenized["subwords"] = tokenized_words
    return tokenized

def tokenize_nolabel_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]

    tokenized["subwords"] = tokenized_words
    return tokenized

#tiny_train_tokenized = tiny_train.map(tokenize_func, batched=True, batch_size=50)
#tiny_test_tokenized = tiny_test.map(tokenize_nolabel_func, batched=True, batch_size=50)

train_tokenized = train.map(tokenize_func, batched=True, batch_size=50)
val_tokenized = val.map(tokenize_func, batched=True, batch_size=50)
test_tokenized = test.map(tokenize_nolabel_func, batched=True, batch_size=50)


  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [ ]:
'''# train for 1 epochs, evaluate after each epoch
training_args = TrainingArguments(
    "test_trainer",
    num_train_epochs=3,
    evaluation_strategy="epoch"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
)
trainer.train()'''

'# train for 1 epochs, evaluate after each epoch\ntraining_args = TrainingArguments(\n    "test_trainer",\n    num_train_epochs=3,\n    evaluation_strategy="epoch"\n)\n\n\ntrainer = Trainer(\n    model=model,\n    args=training_args,\n    train_dataset=tokenized_train,\n    eval_dataset=tokenized_dev,\n)\ntrainer.train()'

In [ ]:
training_args = TrainingArguments(
    output_dir = "content/drive/MyDrive/e_ML4NLP2/"
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
model = BertForTokenClassification.from_pretrained(model, num_labels=labels.num_classes)
trainer = Trainer(model=model, args=training_args, train_dataset=train_tokenized)
trainer.train()

loading configuration file https://huggingface.co/EMBEDDIA/finest-bert/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fb6c00e85cae7f09e35e049995692aa840e064440810b07cf9d1e4df9fab8809.59020bf6435c9bad1a5ae6edb07e5cb3adf696afdc97b518196ad24afdde8014
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_eps": 1e-12

Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

storing https://huggingface.co/EMBEDDIA/finest-bert/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/cdb864d3b4a916a107c261742a540b2ca1bb3172431625ea9bf471790d37a977.a3dfd871932cc7e781255844db2351960c1ebf874975045b7f15bb7e4a5eb636
creating metadata file for /root/.cache/huggingface/transformers/cdb864d3b4a916a107c261742a540b2ca1bb3172431625ea9bf471790d37a977.a3dfd871932cc7e781255844db2351960c1ebf874975045b7f15bb7e4a5eb636
loading weights file https://huggingface.co/EMBEDDIA/finest-bert/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/cdb864d3b4a916a107c261742a540b2ca1bb3172431625ea9bf471790d37a977.a3dfd871932cc7e781255844db2351960c1ebf874975045b7f15bb7e4a5eb636
Some weights of the model checkpoint at EMBEDDIA/finest-bert were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=429, training_loss=0.08484773257951359, metrics={'train_runtime': 155.0257, 'train_samples_per_second': 22.1, 'train_steps_per_second': 2.767, 'total_flos': 174856921164000.0, 'train_loss': 0.08484773257951359, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(test_tokenized)

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subwords, id, words, ner. If subwords, id, words, ner are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 391
  Batch size = 8


In [ ]:
outfile = open("/content/drive/MyDrive/e_ML4NLP2/teamname_bundle4_de_0.tsv", "w")
outfile.write("TOKEN\tNE-COARSE-LIT\tNE-COARSE-METO\tNE-FINE-LIT\tNE-FINE-METO\tNE-FINE-COMP\tNE-NESTED\tNEL-LIT\tNEL-METO\tMISC")

for pred, tinput in zip(predictions.predictions, test_tokenized):
    pred_word_labels = []

    try:
        sent_len = tinput["input_ids"].index(0)
    except ValueError:
        sent_len = len(tinput["input_ids"])
    # Start at 1 to skip pred for cls token
    i = 1

    for word in tinput['subwords']:
        
        word_score = np.zeros_like(pred[0])
        eos_reached = False
        try:
            for subword in word:
                word_score += pred[i]
                i += 1
            label = labels.int2str(int(np.argmax(word_score)))
            pred_word_labels.append(label)
        except IndexError:
            pred_word_labels.append("O")

    for token, label in zip(tinput['words'], pred_word_labels):
        outfile.write("\n" + f"{token}\t{label}\t" + ('\t'.join(['_'] * 8)))

outfile.close()

In [ ]:
accuracy = datasets.load_metric("accuracy")
f1_metric = datasets.load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    refs = labels.flatten()
    predictions = np.argmax(logits, axis=-1)
    print(logits.shape)
    print(predictions.shape)
    pred = predictions.flatten()
    return {
        "accuracy": accuracy.compute(predictions=pred, references=refs)["accuracy"],
        "f1_micro": f1_metric.compute(predictions=pred, references=refs, average="micro")["f1"],
        "f1_macro": f1_metric.compute(predictions=pred, references=refs, average="macro")["f1"],
    }

In [ ]:
Trainer(
    model=model,
    args=training_args,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
).evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subwords, id, words, ner. If subwords, id, words, ner are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 141
  Batch size = 8


(141, 100, 10)
(141, 100)


{'eval_accuracy': 0.9821276595744681,
 'eval_f1_macro': 0.8329829554379456,
 'eval_f1_micro': 0.9821276595744681,
 'eval_loss': 0.06645732372999191,
 'eval_runtime': 2.2602,
 'eval_samples_per_second': 62.384,
 'eval_steps_per_second': 7.964}